In [1]:
from syft import DataSubjectList

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np

In [3]:
from syft import DataSubjectList

In [4]:
a = np.ones((2, 1, 5))
a

array([[[1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.]]])

In [5]:
b = np.ones((10, 1, 1))
b

array([[[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]]])

In [6]:
len(b)

10

(2, 1, 5) and (3, 1, 1) -> (2, 1, 5) -> (max(shape[0], shape[1]))


if len(a) < len(b) -> we have to create a bigger array
if len(a) == len(b) -> it fits!!!!
if len(a) > len(b) -> append some NaNs at the end of b
(3, 1, 5) and (2, 1, 1) -> (3, 1, 5)
ds1
ds2
NaN



(3, 50, 50) and 

In [7]:
dsl = DataSubjectList.from_objs(np.random.choice([10, 11, 12], (5, 5)))

In [8]:
dsl.one_hot_lookup, dsl.data_subjects_indexed.shape

(array([10, 11, 12]), (5, 5))

dsl1 - > (5, 5)
dsl2 -> (3, 5)
index -> 0

dsl1.data_subjects_indexed.shape -> (1, 5)
dsl2 -> (3, 5)

output -> (3, 5, 5)

INPUT:
array([[1, 0, 1, 2, 0],
       [1, 0, 0, 0, 1],
       [0, 0, 2, 0, 1],
       [2, 0, 2, 1, 0],
       [2, 2, 1, 2, 0]])
      
OUTPUT:
array([[6, 6, 6, 6, 6],
       [1, 0, 0, 0, 1],
       [0, 0, 2, 0, 1],
       [2, 0, 2, 1, 0],
       [2, 2, 1, 2, 0]])

array([[7, 7, 7, 7, 7],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN]])

array([[8, 8, 8, 8, 8],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN]])



INPUT

child = [1, 2, 3]
dsl.dsi.shape != child.shape -> GammaTensor
(max(unique_data_subjects_per_data_point), 1, 3)

child = [1 2, 3
         4, 5, 6]
dsl.dsi.shape == child.shape -> PhiTensor

(0, 0, 0;
1, 1, 1)

(6, 6, 6,
7, 7, 7)

index = 0

In [176]:
def get_output_shape(shape1, shape2):
    if len(shape1) > len(shape2) or len(shape2) - len(shape1) > 1:
        raise NotImplementedError
        
    first_dim = max(shape1[0], shape2[0]) if len(shape1) == len(shape2) else shape2[0]
    other_dims = shape1 if len(shape1) < len(shape2) else shape1[1:]
    output_size = (first_dim, *other_dims)
    return output_size


def insert(dsl1: DataSubjectList, dsl2: DataSubjectList, index) -> DataSubjectList:
    
    output_shape = get_output_shape(dsl1.shape, dsl2.shape)
    
    # TODO: Broadcasting with [1] over multiple indices?
    dsl1_len = dsl1.shape[0] if len(dsl1.shape) == len(dsl2.shape) else 1
    dsl2_len = dsl2.shape[0]
    
    
    if dsl1_len < dsl2_len:
        # We need to create a bigger array
        bigger_array = np.full(output_shape, np.nan)
        
        # Fill current dsl1 values into this bigger_array
        # TODO: Ask Madhava if there's a smarter/vectorized way to do this
        if dsl1_len == 1:
            bigger_array[0] = dsl1.data_subjects_indexed
        else:
            for i in range(dsl1_len):
                bigger_array[i] = dsl1[i]

        output_dsl = DataSubjectList(one_hot_lookup=dsl1.one_hot_lookup, data_subjects_indexed=bigger_array)
        # bigger_array[:, index] = dsl2.data_subjects_indexed.squeeze()
        dsl2_copy = dsl2.copy()
        dsl2_copy.data_subjects_indexed = dsl2_copy.data_subjects_indexed.squeeze()
        output_dsl[:, index] = dsl2_copy
        return output_dsl
        
    elif dsl1_len == dsl2_len:
        # it fits perfectly
        output_dsl = dsl1.copy()
        output_dsl[index] = dsl2
        return output_dsl
    else:
        # add some nans
        extra_nans = np.array([np.nan] * (dsl1_len - dsl2_len) * np.prod(dsl2.shape[1:])).reshape((dsl1_len - dsl2_len, *dsl2.shape[1:]))
        array_to_append = np.concatenate((dsl2.data_subjects_indexed, extra_nans)).squeeze()
        new_dsl = dsl2.copy()
        new_dsl.data_subjects_indexed = array_to_append
        
        output_dsl = dsl1.copy()
        output_dsl[:, index] = new_dsl
        return output_dsl
        

In [177]:
def test(dsl1_shape, dsl2_shape, index) -> DataSubjectList:
    dsl1 = DataSubjectList.from_objs(np.random.choice([80, 90, 100], size=dsl1_shape))
    dsl2 = DataSubjectList.from_objs(np.random.choice([50, 60, 70], size=dsl2_shape))
    # print(dsl1.one_hot_lookup, dsl1.data_subjects_indexed.shape)
    # print(dsl2.one_hot_lookup, dsl2.data_subjects_indexed.shape)
    
    output_dsl = insert(dsl1=dsl1, dsl2=dsl2, index=index)
    # print(output_dsl.data_subjects_indexed)
    return output_dsl

In [178]:
res = test((5, 5), (3, 1, 5), index=0)

In [179]:
res = test((5, 5), (3, 1, 5), index=2)

In [180]:
res = test((5, 5), (3, 1, 5), index=-1)

In [181]:
res = test((3, 5, 5), (2, 1, 5), index=0)

In [182]:
res = test((5, 5), (3, 2, 5), index=[0, 1])

In [183]:
res.one_hot_lookup

array([ 80,  90, 100,  50,  60,  70])